<a href="https://colab.research.google.com/github/zloelias/final-project/blob/master/finetune_kinopoisk_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers
! pip install datasets

In [ ]:
! huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: zloelias
Password: 
Login successful
Your token has been saved to /root/.huggingface/token


In [ ]:
! git config --global credential.helper store

In [ ]:
! curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
! sudo apt-get install git-lfs
! git lfs install

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (3.0.2).
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Git LFS initialized.


In [43]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer



In [44]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else: 
    device = torch.device("cpu")
    
print(device)

cuda:0


In [47]:
from datasets import load_dataset

dataset = load_dataset('zloelias/kinopoisk-reviews')

Using custom data configuration zloelias___kinopoisk-reviews-57272c3fc58c0493


  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/266M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.5M [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/parquet/zloelias___kinopoisk-reviews-57272c3fc58c0493/0.0.0/1638526fd0e8d960534e2155dc54fdff8dce73851f21f031d2fb9c2cf757c121. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def count_classes(dataset, label_key='labels'):
    classes = set()
    dataset['train'].map(lambda x: classes.add(x[label_key]))
    return len(classes)

In [ ]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification

BERT_MODEL = "cointegrated/rubert-tiny2"

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL, padding=True, truncation=True)
model = BertForSequenceClassification.from_pretrained(BERT_MODEL, num_labels = count_classes(dataset))
model.to(device)

loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/16af2afaa4ceaa8d50b689bd4c2f7ef7fe3bfac06c0aac7d82a5c1c72298b62a.cc3312d07ccf88871a3c2b7cb3442138e6785101efead94d9f77e96301cf7f4a
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/20317640533199c6b37a557395cd5ee5fcb8777be7c89bb1314bfd43058b35e9.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/e818c3e83969c6aa46da3b5b2eafe049b197e0e787503bf7c643ec64422a51fc.1cec470309dd45bda58f63ce3bb829fe84e2a93e1fc2857ceff76e77262d7944
loading file https://huggingface.co/cointegrated/rubert-tiny2

  0%|          | 0/760816 [00:00<?, ?ex/s]

loading configuration file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/726139048d10597682731a7a4c0b8ef0382927911bc0f6f050a4f7f0afb04c2a.149cdc07694f3925e290abc5528c57a543bcbc9af955c0202b1028584ad15cb4
Model config BertConfig {
  "_name_or_path": "/gd/MyDrive/models/rubert-tiny-mlm-nli-sentence",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(83828, 312, padding_idx=0)
      (position_embeddings): Embedding(2048, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elemen

In [48]:
from transformers import TrainingArguments, Trainer

batch_size = 8

def preprocess_function(examples, sentence_key='text'):
    return tokenizer(examples[sentence_key], padding=True, truncation=True, max_length=512)

dataset = dataset.map(preprocess_function, batched=True)

#dataset['train'] = dataset['train'].select(list(range(0,1800))).map(preprocess_function, batched=True)
#dataset['test'] = dataset['test'].select(list(range(0,300))).map(preprocess_function, batched=True)

  0%|          | 0/1515 [00:00<?, ?ba/s]

  0%|          | 0/169 [00:00<?, ?ba/s]

In [49]:
metric_name = "accuracy"
dataset_name = "kinopoisk-reviews"
task = "clf"
model_name = BERT_MODEL.split("/")[-1]

batch_size = 32

args = TrainingArguments(
    f"{model_name}-{dataset_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=10,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    push_to_hub=True,
)

from sklearn.metrics import accuracy_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
    }

trainer = Trainer(
    model,
    args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
/content/rubert-tiny2-kinopoisk-reviews-finetuned-clf is already a clone of https://huggingface.co/zloelias/rubert-tiny2-kinopoisk-reviews-finetuned-clf. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, label_name.
***** Running training *****
  Num examples = 1514798
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 320
  Gradient Accumulation steps = 10
  Total optimization steps = 23665


Epoch,Training Loss,Validation Loss,Accuracy
0,0.762500,0.757629,0.702046
1,0.746800,0.750660,0.705700


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, label_name.
***** Running Evaluation *****
  Num examples = 168311
  Batch size = 32
Saving model checkpoint to rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-4733
Configuration saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-4733/config.json
Model weights saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-4733/pytorch_model.bin
tokenizer config file saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-4733/tokenizer_config.json
Special tokens file saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/checkpoint-4733/special_tokens_map.json
tokenizer config file saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/tokenizer_config.json
Special tokens file saved in rubert-tiny2-kinopoisk-reviews-finetuned-clf/special_tokens_map.json
The following columns in the evaluation set  don't h

In [ ]:
! export PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:256